In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# Not sure where the unnamed column came from but just drop it
df = pd.read_csv('../data/processed_data.csv').drop(u'Unnamed: 0', axis=1) 
df = df.dropna()
print(df.shape)
print(df.columns)
df.head()

(25697, 59)
Index([u'Opponent DRtg/A', u'action_type#Alley Oop Dunk Shot',
       u'action_type#Driving Dunk Shot',
       u'action_type#Driving Finger Roll Layup Shot',
       u'action_type#Driving Finger Roll Shot',
       u'action_type#Driving Jump shot', u'action_type#Driving Layup Shot',
       u'action_type#Dunk Shot', u'action_type#Fadeaway Bank shot',
       u'action_type#Hook Shot', u'action_type#Jump Bank Shot',
       u'action_type#Jump Hook Shot', u'action_type#Jump Shot',
       u'action_type#Layup Shot', u'action_type#Pullup Jump shot',
       u'action_type#Putback Slam Dunk Shot',
       u'action_type#Running Finger Roll Shot',
       u'action_type#Running Hook Shot', u'action_type#Running Jump Shot',
       u'action_type#Slam Dunk Shot', u'combined_shot_type#Dunk',
       u'combined_shot_type#Hook Shot', u'combined_shot_type#Jump Shot',
       u'combined_shot_type#Layup', u'combined_shot_type#Tip Shot', u'day#31',
       u'home', u'last_5_seconds', u'loc_x#(-10.96, 8.96

,Opponent DRtg/A,action_type#Alley Oop Dunk Shot,action_type#Driving Dunk Shot,action_type#Driving Finger Roll Layup Shot,action_type#Driving Finger Roll Shot,action_type#Driving Jump shot,action_type#Driving Layup Shot,action_type#Dunk Shot,action_type#Fadeaway Bank shot,action_type#Hook Shot,...,playoffs,season,segababa,shot_distance,shot_zone_range#16-24 ft.,shot_zone_range#24+ ft.,shot_zone_range#Back Court Shot,shot_zone_range#Less Than 8 ft.,year#2016,shot_made_flag
0,108.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,18,1.0,0.0,0.0,0.0,0.0,0.0
1,101.68,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,16,1.0,0.0,0.0,0.0,0.0,0.0
2,109.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,23,1.0,0.0,0.0,0.0,0.0,1.0
3,109.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,0,0.0,0.0,0.0,1.0,0.0,0.0
4,109.65,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,1,13,0.0,0.0,0.0,0.0,0.0,1.0


In [2]:
df_submit = pd.read_csv('../data/submit_data.csv').drop(u'Unnamed: 0', axis=1)
print df_submit.shape

X_submit = df_submit.drop(['shot_id'], axis=1)

(5000, 59)


In [3]:
y = df['shot_made_flag']
X = df.drop('shot_made_flag', axis=1)

## Some more initializations...

In [32]:
from sklearn.model_selection import KFold
seed = 7
processors=1
num_folds = 3
scoring='neg_log_loss'
kfold = KFold(n_splits=num_folds, random_state=seed)

## Use this function to create a submission

In [33]:
def create_submission(model_grid):
    predictions = model_grid.predict_proba(X_submit)
    submission = pd.DataFrame()
    submission["shot_id"] = df_submit['shot_id']
    submission["shot_made_flag"] = predictions.T[1]
    submission = submission.sort_values(['shot_id'], ascending=True)
    return submission

## XGBoost

In [6]:
# Change the columns to new column names for XGBoost
X_xgb = X
X_xgb.columns = list(range(len(X.columns)))

In [33]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

cv_params = {'max_depth': [3,5,7], 'min_child_weight': [1,3,5]}
ind_params = {'learning_rate': 0.1, 'n_estimators': 1000, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic'}

xg_grid = GridSearchCV(
    xgb.XGBClassifier(**ind_params),
    cv_params,
    scoring = scoring, 
    cv=kfold,
    n_jobs=processors)


xg_grid.fit(X_xgb, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.8),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 7], 'min_child_weight': [1, 3, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

### Create a submission

In [35]:
X_submit_xgb = X_submit
X_submit_xgb.columns = list(range(len(X_submit.columns)))
predictions = xg_grid.predict_proba(X_submit_xgb)
submission = pd.DataFrame()
submission["shot_id"] = df_submit['shot_id']
submission["shot_made_flag"] = predictions.T[1]
submission = submission.sort_values(['shot_id'], ascending=True)
submission.head()

,shot_id,shot_made_flag
694,1,0.318867
695,8,0.287643
697,17,0.488713
696,20,0.590384
705,33,0.441181


In [36]:
submission.to_csv("../data/submission_XGBoost_proba.csv",index=False)

## Linear Discriminant Analysis

In [6]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV

lda_grid = GridSearchCV(
    estimator = LinearDiscriminantAnalysis(),
    param_grid = {
        'solver': ['lsqr'],
        'shrinkage': [0, 0.25, 0.5, 0.75, 1],
        'n_components': [None, 2, 5, 10]
    },
    cv = kfold,
    scoring = scoring,
    n_jobs = processors)

lda_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
              solver='svd', store_covariance=False, tol=0.0001),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'shrinkage': [0, 0.25, 0.5, 0.75, 1], 'n_components': [None, 2, 5, 10], 'solver': ['lsqr']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [7]:
print lda_grid.best_params_

{'shrinkage': 0, 'n_components': None, 'solver': 'lsqr'}


### Create a submission

In [11]:
submission = create_submission(lda_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.326687
695,8,0.255678
697,17,0.649706
696,20,0.765392
705,33,0.335131


In [12]:
submission.to_csv("../data/submission_lda.csv",index=False)

## Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

lr_grid = GridSearchCV(
    estimator = LogisticRegression(random_state=seed),
    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.001, 0.01, 1, 10, 100, 1000],        
    },
    cv = kfold,
    scoring = scoring,
    n_jobs = processors)

lr_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=7, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [9]:
print lr_grid.best_params_

{'penalty': 'l2', 'C': 1}


### Create a submission

In [14]:
submission = create_submission(lr_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.347909
695,8,0.277378
697,17,0.613634
696,20,0.748740
705,33,0.357666


In [15]:
submission.to_csv("../data/submission_log_r.csv",index=False)

## Random Forest

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

rf_grid = GridSearchCV(
    estimator = RandomForestClassifier(warm_start=True, random_state=seed),
    param_grid = {
        'n_estimators': [100, 200],
        'criterion': ['gini', 'entropy'],
        'max_depth': [8, 10],
        'bootstrap': [True],
        'max_features': [18, 20]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

rf_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=7,
            verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200], 'bootstrap': [True], 'criterion': ['gini', 'entropy'], 'max_depth': [8, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [11]:
print rf_grid.best_params_

{'n_estimators': 200, 'bootstrap': True, 'criterion': 'gini', 'max_depth': 10}


In [17]:
submission = create_submission(rf_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.356164
695,8,0.370993
697,17,0.666709
696,20,0.699689
705,33,0.368841


In [18]:
submission.to_csv("../data/submission_rf.csv",index=False)

## Gradient Boosting

In [6]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


gbm_grid = GridSearchCV(
    estimator = GradientBoostingClassifier(warm_start=True, random_state=seed),
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [2, 3, 4],
        'learning_rate': [1e-1, 1]
    }, 
    cv = kfold, 
    scoring = scoring, 
    n_jobs = processors)

gbm_grid.fit(X, y)

GridSearchCV(cv=KFold(n_splits=3, random_state=7, shuffle=False),
       error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_split=1e-07, min_samples_leaf=1,
              min_samples_split=2, min_weight_fraction_leaf=0.0,
              n_estimators=100, presort='auto', random_state=7,
              subsample=1.0, verbose=0, warm_start=True),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [100, 200], 'learning_rate': [0.1, 1], 'max_depth': [2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_log_loss', verbose=0)

In [7]:
print gbm_grid.best_params_

{'n_estimators': 100, 'learning_rate': 0.1, 'max_depth': 3}


In [8]:
submission = create_submission(gbm_grid)
submission.head()

,shot_id,shot_made_flag
694,1,0.369379
695,8,0.304920
697,17,0.682503
696,20,0.750638
705,33,0.360407


In [9]:
submission.to_csv("../data/submission_gbc.csv",index=False)

## Creating a voting ensemble

In [34]:
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
# Create sub models
estimators = []

estimators.append(('lr', LogisticRegression(penalty='l2', C=1)))
estimators.append(('gbm', GradientBoostingClassifier(n_estimators=100, max_depth=3, learning_rate=0.1, max_features=15, warm_start=True, random_state=seed)))
estimators.append(('rf', RandomForestClassifier(bootstrap=True, max_depth=10, n_estimators=200, max_features=20, criterion='entropy', random_state=seed)))

# create the ensemble model
ensemble = VotingClassifier(estimators, voting='soft', weights=[2,3,3])

ensemble.fit(X, y)

VotingClassifier(estimators=[('lr', LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('gbm', Gradient...stimators=200, n_jobs=1, oob_score=False, random_state=7,
            verbose=0, warm_start=False))],
         n_jobs=1, voting='soft', weights=[2, 3, 3])

In [35]:
submission = create_submission(ensemble)
submission.head()

,shot_id,shot_made_flag
694,1,0.363395
695,8,0.322246
697,17,0.647568
696,20,0.722640
705,33,0.363262


In [36]:
submission.to_csv("../data/submission_ensemble.csv", index=False)